# Funciones requeridas para la API

En esta Jupyter Notebook se desarrollan y prueban las funciones que son solicitadas para la API. Luego, al momento del deployarlas es posible que sufran algunos cambios en función de los requerimientos propios de FastAPI o Render.

## Importaciones

In [1]:
import pandas as pd

## Carga de los dataset necesarios

Se cargan los conjuntos de datos necesarios para llevar a cabo las consultas solicitadas. Se decide subir los datos de esta manera, y no los dataset completos, debido a la limitada capacidad de almacenamiento que permite Render. Por otra parte, considerando que el objetivo es entregar un PVM, a modo de prueba de concepto, se consideró adecuada esta simplificación de los datasets.

Los  dataset que se utilizan son:

* **games**: Contiene la información de los juegos de Steam, entre ellos Devs, Publicistas, Género, etc.
* **reviews**: Contiene la información relacionada a los usuarios que realizan reviews de los juegos en formato de análisis de sentimientos. Entre esta información, se encuentran las recomendaciones o no del juego por parte de usuario, la fecha del review así como datos del usuario como su id, su url del perfil.
* **gasto_items**: Contiene la cantidad de items que consume cada usuario y el precio de cada uno de los productos que consume.
* **user_time_year**: Contiene para cada usuario su identificación, la url de su perfil, la cantidad de horas jugadas por cada género de juego.
* **ranking_genero**: Contiene el ranking de los géneros de juegos con mas horas jugadas.
* **items_developer**: Contiene la información relacionada con cada item de juegos como su id, precio, desarrollador y año de lanzamiento.
* **top_dev**: Contiene la información de los desarrolladores junto con las recomendaciones, el análisis de sentimiento y el año de lanzamiento

In [2]:
games = pd.read_parquet('data/01-steam-games.parquet')
reviews = pd.read_parquet('data/02-user-reviews.parquet')
gasto_items = pd.read_parquet('data/04-gasto-items.parquet')
ranking_genero = pd.read_parquet('data/05-ranking-genero.parquet')
items_developer = pd.read_parquet('data/07-items-developer.parquet')
user_time_year = pd.read_parquet('data/08-user-time-year.parquet')
top_dev = pd.read_parquet('data/09-top-dev.parquet')

A continuación, se desarrollan cada una de las funciones solicitadas.

## Función `developer`

Esta función recibe como parámetro 'developer', que es la empresa desarrolladora del juego, y devuelve la cantidad de items que desarrolla dicha empresa y el porcentaje de contenido Free por año por sobre el total que desarrolla

In [3]:
def developer(desarrollador):
    """
        Esta función retorna la cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

    Args:
        def developer( desarrollador : str ):

    Returns:
        dict: {'release_year': {'cantidad_juegos': int,
                'cantidad_gratis': int,
                'porcentaje_gratis': int}}
    """
    # Filtra el dataframe por desarrollador de interés
    filtro = items_developer[items_developer['developer'] == desarrollador]
    # Calcula la cantidad de items y la cantidad de elementos gratis por año
    resumen_por_año = filtro.groupby('release_year').agg(cantidad_juegos=('item_id', 'count'),
                                                                cantidad_gratis=('price', lambda x: (x == 0.0).sum()))
    
    # Calcula el porcentaje de elementos gratis por año
    resumen_por_año['porcentaje_gratis'] = (resumen_por_año['cantidad_gratis'] / resumen_por_año['cantidad_juegos'] * 100).fillna(0).astype(int)

    return resumen_por_año.to_dict(orient='index')

In [4]:
desarrollador = 'Secret Level SRL'
developer(desarrollador)

{2018: {'cantidad_juegos': 1, 'cantidad_gratis': 1, 'porcentaje_gratis': 100}}

## Función `userdata`

Esta función tiene por parámentro 'user_id' y devulve la cantidad de dinero gastado por el usuario, el porcentaje de recomendaciones que realizó sobre la cantidad de reviews que se analizan y la cantidad de items que consume el mismo.

In [5]:
def userdata(user_id, total_reviews=None):
    """
    Esta función devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews_recommend y cantidad de items.

    Parámetros:
    - user_id (int): Identificador único del usuario de interés.
    - total_reviews (int, opcional): Total de revisiones realizadas por todos los usuarios. Si no se proporciona,
      se calculará como la longitud de los identificadores únicos de usuarios en el conjunto de datos de revisiones.

    Devuelve:
    dict: 
        - 'usuario_': Identificador único del usuario.
        - 'cantidad_dinero': Suma total de dinero gastado por el usuario en gasto_items.
        - 'porcentaje_recomendacion': Porcentaje de recomendaciones realizadas por el usuario en comparación
          con el total de revisiones en el conjunto de datos.
        - 'total_items': La cantidad máxima de items comprados por el usuario en gasto_items.
    """
    # Filtra por el usuario de interés en gasto_items
    usuario_gastos = gasto_items[gasto_items['user_id'] == user_id]
    
    # Calcula la cantidad de dinero gastado y el total de items para el usuario de interés
    
    cantidad_dinero = usuario_gastos['price'].sum()
    count_items = usuario_gastos['items_count'].max()

    # Calcula el total de recomendaciones realizadas por el usuario de interés
    total_recomendaciones = reviews[reviews['user_id'] == user_id]['reviews_recommend'].sum()

    # Calcula el porcentaje de recomendaciones realizadas por el usuario de interés
    if total_reviews is None:
        total_reviews = len(reviews['user_id'].unique())

    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100

    return {
        'usuario_': user_id,
        'cantidad_dinero': cantidad_dinero,
        'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
        'total_items': count_items
    }



In [6]:
user_id = 'evcentric'
userdata(user_id)

{'usuario_': 'evcentric',
 'cantidad_dinero': 1387.15,
 'porcentaje_recomendacion': 0.02,
 'total_items': 137}

## Función `UserForGenre`

Esta función recibe como parámetro un género de videojuego y devuelve el puesto en el que se encuentra dicho género sobre un ranking de los mismos analizando la cantidad de horas jugadas para cada uno.

In [7]:
def UserForGenre(genero, usuario_especifico=None):
    """
    Devuelve el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

    Parámetros:
    - df (DataFrame): DataFrame que contiene las columnas 'playtime_forever', 'user_id', 'genres', 'release_year'.
    - genero (str): Género específico para el cual se desea obtener la información.
    - usuario_especifico (str, opcional): Usuario específico para el cual se calculará la acumulación de horas jugadas. Si no se proporciona, se usará el usuario con más horas jugadas.

    Retorna:
    dict: Un diccionario con la siguiente estructura:
        {
            "Usuario con más horas jugadas para <genero>": <usuario>,
            "Horas jugadas": [
                {"Año": <año1>, "Horas": <horas1>},
                {"Año": <año2>, "Horas": <horas2>},
                ...
            ]
        }
    """

    # Filtra el DataFrame por el género dado
    data_genero = user_time_year[user_time_year['genres'].str.contains(genero, case=False, na=False)]

    if usuario_especifico is None:
        # Encuentra el usuario con más horas jugadas para el género dado
        usuario_mas_horas = data_genero.groupby('user_id')['playtime_forever'].sum().idxmax()
    else:
        usuario_mas_horas = usuario_especifico

    # Filtra nuevamente por el usuario con más horas jugadas
    data_genero_usuario = data_genero[data_genero['user_id'] == usuario_mas_horas]

    # Calcula la acumulación de horas jugadas por año de lanzamiento
    horas_por_año = data_genero_usuario.groupby('release_year')['playtime_forever'].sum().reset_index()

    # Formatea el resultado como un diccionario
    resultado = {
        "Usuario con más horas jugadas para {}: {}".format(genero, usuario_mas_horas): usuario_mas_horas,
        "Horas jugadas": [{"Año": int(año), "Horas": int(horas)} for año, horas in zip(horas_por_año['release_year'], horas_por_año['playtime_forever'])]
    }

    return resultado



In [9]:
# Ejemplo de uso
genero = "Indie"
resultado_ejemplo = UserForGenre(genero)
resultado_ejemplo


{'Usuario con más horas jugadas para Indie: 76561198055326819': '76561198055326819',
 'Horas jugadas': [{'Año': 2006, 'Horas': 5748}]}

## Función `best_developer_year`

In [14]:
def UsersRecommend(año):
    """
    Devuelve el top 3 de desarrolladores con juegos MÁS recomendados y análisis de sentimientos positivos (2) por usuarios para el año dado.

    Parámetros:
    - año (int): Año específico para el cual se desea obtener la información.

    Retorna:
    list: Una lista de diccionarios con el siguiente formato:
        [
            {"Puesto 1": <desarrollador1>, "recomendaciones": <cantidad1>},
            {"Puesto 2": <desarrollador2>, "recomendaciones": <cantidad2>},
            {"Puesto 3": <desarrollador3>, "recomendaciones": <cantidad3>}
        ]
    """
    # Filtramos por el año deseado
    df_filtered = top_dev[top_dev['release_year'] == año]

    # Filtramos por comentarios recomendados y sentiment_analysis positivo/neutral
    df_filtered = df_filtered[(df_filtered['reviews_recommend'] == True) & (df_filtered['sentiment_analysis'].isin([2]))]

    # Obtenemos el top 3 de juegos recomendados
    top_games = df_filtered['developer'].value_counts().head(3).reset_index()

    # Modificamos la estructura del resultado
    result = [{"Puesto {}".format(i + 1): juego, 'recomendaciones': count} for i, (juego, count) in enumerate(zip(top_games['developer'], top_games['count']))]

    return result

In [16]:
# Ejemplo de uso
año_ejemplo = 2012
resultado_ejemplo = UsersRecommend(año_ejemplo)
resultado_ejemplo


[{'Puesto 1': 'The Behemoth', 'recomendaciones': 3},
 {'Puesto 2': 'Daedalic Entertainment', 'recomendaciones': 3},
 {'Puesto 3': 'Flying Wild Hog', 'recomendaciones': 2}]

## Función `developer_reviews_analysis`

In [12]:
def developer_reviews_analysis(desarrollador):
    """
    Analiza las reseñas de usuarios para un desarrollador específico y devuelve un diccionario con la cantidad total
    de registros de reseñas categorizados con un análisis de sentimiento como positivo o negativo.

    Parámetros:
    - df (DataFrame): DataFrame que contiene las columnas 'playtime_forever', 'user_id', 'item_id', 'genres', 'release_year'.
    - desarrollador (str): Nombre del desarrollador para el cual se desea realizar el análisis.

    Retorna:
    dict: Un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de
    reseñas categorizados con un análisis de sentimiento como valor positivo o negativo.
    """
    # Filtra el DataFrame por el desarrollador deseado
    df_desarrollador = top_dev[top_dev['developer'] == desarrollador]

    # Cuenta la cantidad total de registros de reseñas categorizados como positivos o negativos
    count_sentimiento = df_desarrollador['sentiment_analysis'].value_counts()

    # Formatea el resultado como un diccionario
    result = {desarrollador: {'Negative': count_sentimiento.get(0, 0), 'Positive': count_sentimiento.get(2, 0)}}

    return result



In [13]:
# Ejemplo de uso
desarrollador_ejemplo = "Valve"
resultado_ejemplo = developer_reviews_analysis(desarrollador_ejemplo)
resultado_ejemplo

{'Valve': {'Negative': 30, 'Positive': 35}}